<a href="https://colab.research.google.com/github/esequielsantos/ediBot/blob/develop/EdiBot.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#EdiBot
###Segue requisitos e uso da API
Instalar Gemini

---



In [1]:
!pip install -U google-generativeai

## Set up my API key

To use the Gemini API, you'll need an API key. If you don't already have one, create a key in Google AI Studio.

<a class="button" href="https://aistudio.google.com/app/apikey" target="_blank" rel="noopener noreferrer">Get an API key</a>

In Colab, add the key to the secrets manager under the "🔑" in the left panel. Give it the name `GOOGLE_API_KEY`. Then pass the key to the SDK:


In [2]:
import google.generativeai as genai
# Used to securely store your API key
from google.colab import userdata

GOOGLE_API_KEY=userdata.get('GOOGLE_API_KEY')
genai.configure(api_key=GOOGLE_API_KEY)

Listar modelos disponiveis


In [3]:
for m in genai.list_tuned_models():
  print (m)

Unauthorized: 401 GET https://generativelanguage.googleapis.com/v1beta/tunedModels?pageSize=50&%24alt=json%3Benum-encoding%3Dint: API keys are not supported by this API. Expected OAuth2 access token or other authentication credentials that assert a principal. See https://cloud.google.com/docs/authentication

In [4]:
for m in genai.list_models():
  if 'generateContent' in m.supported_generation_methods:
    print(m.name)

models/gemini-1.0-pro
models/gemini-1.0-pro-001
models/gemini-1.0-pro-latest
models/gemini-1.0-pro-vision-latest
models/gemini-1.5-pro-latest
models/gemini-pro
models/gemini-pro-vision


Configurando o modelo
Temperaura:

In [5]:
gen_config = {
    "candidate_count": 1,
    "temperature": 1,
}

Segurança:

In [6]:
saf_settings = {
    "HARASSMENT": "BLOCK_NONE",
    "HATE": "BLOCK_NONE",
    "SEXUAL": "BLOCK_NONE",
    "DANGEROUS": "BLOCK_NONE",
}

Usar **modelo** escolhido acima

In [7]:
model = genai.GenerativeModel(model_name='gemini-1.0-pro-latest', safety_settings= saf_settings, generation_config= gen_config)

#Enviando mensagem ao modelo

In [ ]:
response = model.generate_content("Estou programndo em Phyton. Sugira 5 opções indispensáveis de conhecermos no aprendizado de Phyton")
#print(model.generate_content("Estou programndo em Phyton. Sugira 5 opções indispensáveis de conhecermos no aprendizado de Phyton").text)

##Exibindo Resposta

In [ ]:
print(response.text)

NameError: name 'response' is not defined

#EdiBOT

> Adicionar aspas INICIANDO O EdiBot

**Iniciando**



In [8]:
from datetime import datetime

In [9]:
chat = model.start_chat(history=[])

In [12]:
def edibot(prompt, history):
  if not chat.history:
    agora = datetime.now()
    agorastr= formatted_string = f"{agora:%d/%m/%Y %H:%M:%S}"
    hora = agora.hour
    periodo_dia = ""
    if 6 <= hora < 12:
      periodo_dia = "manhã"
    elif 12 <= hora < 18:
      periodo_dia = "tarde"
    else:
      periodo_dia = "noite"
    bemvindo = model.generate_content(f"Você é um atendente de uma Loja física de informatica chamada ByteMe, que vende computadores, laptops e acessórios chamado EdiBot. Escreva uma mensagem de saudação de {periodo_dia} ").text
    response = chat.send_message(prompt)
    return agorastr + '\n\n' + bemvindo + '\n\n-----------------------\n' + response.text
  else:
    prompt = prompt.lower()
    agradecer = model.generate_content("Você é um atendente de uma empresa de informatica chamada ByteMe. Escreva uma mensagem curta de agradecimento por ter entrado em contato").text
    if prompt == "history":
      markdown_text = ""
      for message in chat_history:
        sender = message.sender
        text = message.text
        markdown_text += f"**{sender}**: {text}\n"
      return markdown_text
    elif 'obrigado' in prompt:
      return agradecer
    elif 'tchau' in prompt:
      return agradecer
    elif 'fim' in prompt:
      return agradecer
    elif 'sair' in prompt:
      return agradecer
    else:
      response = chat.send_message(prompt)
      return response.text
  #continuar = model.generate_content("Escreva uma mensagem curta e agradavél perguntando se deseja mais alguma informação").text
  #return continuar

##Teste da funcao inteligente
*Exige parada total do GRadio*

In [11]:
print(edibot("Bom dia, preciso de um computador novo para jogar", []))

10-05-2024 21:31:55
"Boa noite, seja muito bem-vindo à ByteMe!

Meu nome é [Seu Nome] e sou o atendente que irá lhe auxiliar neste momento.

Estamos aqui para ajudá-lo a encontrar o computador, laptop ou acessório EdiBot perfeito para suas necessidades. Sinta-se à vontade para explorar nossa ampla seleção e nos perguntar qualquer dúvida que possa ter.

-----------------------
**Configurações Recomendadas para Jogos:**

* **Processador:** Intel Core i5-12400 ou AMD Ryzen 5 5600X
* **Placa de Vídeo:** NVIDIA GeForce RTX 3060 ou AMD Radeon RX 6600 XT
* **Memória (RAM):** 16 GB (2 x 8 GB) DDR4-3200
* **Armazenamento:** SSD NVMe de 512 GB ou 1 TB
* **Fonte de Alimentação:** 600W 80+ Bronze

**Configurações Avançadas para Jogos:**

* **Processador:** Intel Core i7-12700 ou AMD Ryzen 7 5800X
* **Placa de Vídeo:** NVIDIA GeForce RTX 3070 ou AMD Radeon RX 6700 XT
* **Memória (RAM):** 32 GB (4 x 8 GB) DDR4-3600
* **Armazenamento:** SSD NVMe de 1 TB ou 2 TB
* **Fonte de Alimentação:** 750W 80+ Go

In [13]:
print(edibot("e se for pra usar Autocad ?", []))

**Configurações Recomendadas para AutoCAD:**

* **Processador:** Intel Core i7 de 8ª geração ou AMD Ryzen 7 de 2ª geração
* **Placa de Vídeo:** NVIDIA Quadro P620 ou AMD Radeon Pro WX 3200
* **Memória (RAM):** 16 GB (2 x 8 GB) DDR4-2666
* **Armazenamento:** SSD NVMe de 512 GB ou 1 TB
* **Fonte de Alimentação:** 500W 80+ Bronze

**Configurações Avançadas para AutoCAD:**

* **Processador:** Intel Core i9 de 11ª geração ou AMD Ryzen 9 de 3ª geração
* **Placa de Vídeo:** NVIDIA Quadro RTX 4000 ou AMD Radeon Pro WX 8200
* **Memória (RAM):** 32 GB (4 x 8 GB) DDR4-3200
* **Armazenamento:** SSD NVMe de 1 TB ou 2 TB
* **Fonte de Alimentação:** 650W 80+ Gold

**Considerações Adicionais:**

* **Monitor:** 24-27 polegadas com resolução de pelo menos 1920x1080
* **Mouse:** Com botões programáveis para atalhos do AutoCAD
* **Sistema Operacional:** Windows 11 Pro ou Windows 10 Pro

**Sugestões de Modelos:**

* **Opção Econômica:** HP Z240 Workstation (Processador Intel Core i7-8700, Placa de Vídeo NV

In [ ]:
#print(edibot('history', []))
chat.history

##Uso do GRADIO

In [14]:
!pip install -U gradio

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.3/12.3 MB 73.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 92.0/92.0 kB 13.3 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 315.5/315.5 kB 29.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 75.6/75.6 kB 9.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 142.5/142.5 kB 19.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.7/8.7 MB 94.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 47.2/47.2 kB 6.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 60.8/60.8 kB 7.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 129.9/129.9 kB 16.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 77.9/77.9 kB 10.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.3/58.3 kB 8.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 71.9/71.9 kB 9.7 MB/s 

In [15]:
import gradio as gr

#Tela Gráfica para o BOT

In [ ]:
demo = gr.ChatInterface(edibot,
                 title='BatePapo BiteMe',
                 textbox=gr.Textbox(placeholder="Pergunte ao EdiBOT"),
                 retry_btn='Voltar',
                 clear_btn='Limpar',
                 undo_btn='Desfazer',
                 stop_btn='Parar',
                 submit_btn='Enviar [Enter]')

if __name__ == "__main__":
    demo.launch(debug=True)

Setting queue=True in a Colab notebook requires sharing enabled. Setting `share=True` (you can turn this off by setting `share=False` in `launch()` explicitly).

Colab notebook detected. This cell will run indefinitely so that you can see errors and logs. To turn off, set debug=False in launch().
Running on public URL: https://8263c74eb1ed67796a.gradio.live

This share link expires in 72 hours. For free permanent hosting and GPU upgrades, run `gradio deploy` from Terminal to deploy to Spaces (https://huggingface.co/spaces)


Traceback (most recent call last):
  File "/usr/local/lib/python3.10/dist-packages/gradio/queueing.py", line 527, in process_events
    response = await route_utils.call_process_api(
  File "/usr/local/lib/python3.10/dist-packages/gradio/route_utils.py", line 270, in call_process_api
    output = await app.get_blocks().process_api(
  File "/usr/local/lib/python3.10/dist-packages/gradio/blocks.py", line 1897, in process_api
    data = await self.postprocess_data(fn_index, result["prediction"], state)
  File "/usr/local/lib/python3.10/dist-packages/gradio/blocks.py", line 1723, in postprocess_data
    prediction_value = block.postprocess(prediction_value)
  File "/usr/local/lib/python3.10/dist-packages/gradio/components/chatbot.py", line 230, in postprocess
    self._postprocess_chat_messages(message_pair[1]),
  File "/usr/local/lib/python3.10/dist-packages/gradio/components/chatbot.py", line 195, in _postprocess_chat_messages
    return FileMessage(
  File "/usr/local/lib/python3.10/dis

Subir para o GitHub